In [22]:
pip install -q torchtext

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.1 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.15.1 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:
import torch
from torch import nn
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torchvision.transforms import Resize, ToTensor, Normalize
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as tt
import json
import re
import string
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from torchtext.vocab import Vocab
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

import matplotlib.pyplot as plt
import numpy as np
#from vae_utils import get_vector_from_label, add_vector_to_images, morph_faces

print(torch.__version__)
print(torchvision.__version__)

device = torch.device("cuda:0" if torch.cuda.is_available() else "mps")

2.0.0
0.15.1


In [24]:
VOCAB_SIZE = 10000
MAX_LEN = 200
EMBEDDING_DIM = 100
N_UNITS = 128
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 25

In [10]:
import zipfile

zip_file = '/Users/parkermoesta/Generative_models/Generative_models/Autoregressive Models/LSTM/epirecipes.zip'
extract_dir = '/Users/parkermoesta/Generative_models/Generative_models/Autoregressive Models/LSTM/'

# Open the zip file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    # Extract all the contents of the zip file to the specified directory
    zip_ref.extractall(extract_dir)

print("Extraction complete.")

Extraction complete.


## Load the data

In [13]:
# Load the full dataset
with open("/Users/parkermoesta/Generative_models/Generative_models/Autoregressive Models/LSTM/full_format_recipes.json") as json_data:
    recipe_data = json.load(json_data)

In [15]:
# Filter the dataset
filtered_data = [
    "Recipe for " + x["title"] + " | " + " ".join(x["directions"])
    for x in recipe_data
    if "title" in x
    and x["title"] is not None
    and "directions" in x
    and x["directions"] is not None
]

In [16]:
# Count the recipes
n_recipes = len(filtered_data)
print(f"{n_recipes} recipes loaded")

20111 recipes loaded


In [25]:
example = filtered_data[9]
print(example)

Recipe for Ham Persillade with Mustard Potato Salad and Mashed Peas  | Chop enough parsley leaves to measure 1 tablespoon; reserve. Chop remaining leaves and stems and simmer with broth and garlic in a small saucepan, covered, 5 minutes. Meanwhile, sprinkle gelatin over water in a medium bowl and let soften 1 minute. Strain broth through a fine-mesh sieve into bowl with gelatin and stir to dissolve. Season with salt and pepper. Set bowl in an ice bath and cool to room temperature, stirring. Toss ham with reserved parsley and divide among jars. Pour gelatin on top and chill until set, at least 1 hour. Whisk together mayonnaise, mustard, vinegar, 1/4 teaspoon salt, and 1/4 teaspoon pepper in a large bowl. Stir in celery, cornichons, and potatoes. Pulse peas with marjoram, oil, 1/2 teaspoon pepper, and 1/4 teaspoon salt in a food processor to a coarse mash. Layer peas, then potato salad, over ham.


## Tokenization

In [18]:
# Pad the punctuation, to treat them as separate 'words'
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


text_data = [pad_punctuation(x) for x in filtered_data]
text_data[9]

'Recipe for Ham Persillade with Mustard Potato Salad and Mashed Peas | Chop enough parsley leaves to measure 1 tablespoon ; reserve . Chop remaining leaves and stems and simmer with broth and garlic in a small saucepan , covered , 5 minutes . Meanwhile , sprinkle gelatin over water in a medium bowl and let soften 1 minute . Strain broth through a fine - mesh sieve into bowl with gelatin and stir to dissolve . Season with salt and pepper . Set bowl in an ice bath and cool to room temperature , stirring . Toss ham with reserved parsley and divide among jars . Pour gelatin on top and chill until set , at least 1 hour . Whisk together mayonnaise , mustard , vinegar , 1 / 4 teaspoon salt , and 1 / 4 teaspoon pepper in a large bowl . Stir in celery , cornichons , and potatoes . Pulse peas with marjoram , oil , 1 / 2 teaspoon pepper , and 1 / 4 teaspoon salt in a food processor to a coarse mash . Layer peas , then potato salad , over ham . '

In [38]:
class TextDataset(Dataset):
    def __init__(self, text_data, tokenizer):
        self.text_data = text_data
        self.tokenizer = tokenizer
    
    def __getitem__(self, idx):
        tokens = self.tokenizer(self.text_data[idx])
        tokens = tokens[:200]  # Clip to maximum length of 200
        return tokens
    
    def __len__(self):
        return len(self.text_data)

In [39]:
tokenizer = get_tokenizer('basic_english')

In [40]:
def yield_tokens(data_iter):
    for text_str in data_iter:
        yield tokenizer(text_str)

In [41]:
vocab = build_vocab_from_iterator(yield_tokens(text_data), max_tokens=VOCAB_SIZE, specials=['<pad>', '<unk>'])
vocab.set_default_index(vocab['<unk>'])

In [53]:
def collate_batch(batch):
    batch = [torch.tensor([vocab[token] for token in tokens if token in vocab.get_stoi()]) for tokens in batch]
    batch = pad_sequence(batch, batch_first=True, padding_value=vocab['<pad>'])
    x = batch[:, :-1]
    y = batch[:, 1:]
    return x, y



In [54]:
text_ds = TextDataset(text_data, tokenizer)
text_loader = DataLoader(text_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

In [60]:
# Get the first batch from the DataLoader
for x, y in text_loader:
    break

# Convert indices back to tokens
def indices_to_tokens(indices):
    itos = vocab.get_itos()
    return ' '.join([itos[index] for index in indices])

# Print the first few sentences in the batch
for i in range(5):  # Change this number to print more or fewer sentences
    print("Input (x):", indices_to_tokens(x[i]))
    print("Target (y):", indices_to_tokens(y[i]))
    print("---")

Input (x): recipe for spiced green tahini sauce | pulse garlic , cilantro , parsley , cumin , fenugreek , and 2 teaspoons salt in a food processor until similar in texture to pesto . add tahini and lemon juice process 30 seconds ( mixture will be very thick and gray ) . with motor running , gradually drizzle in 3 / 4 cup water and process , adding more water to thin if needed , until sauce is light green and the consistency of sour cream . season with salt . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa

In [59]:
for x, y in text_loader:
    # x and y are the input and target sequences for this batch
    break

x, y

(tensor([[  25,   16,  477,  ...,    0,    0,    0],
         [  25,   16, 1489,  ...,    0,    0,    0],
         [  25,   16, 1268,  ...,  174,   63,    6],
         ...,
         [  25,   16, 2002,  ...,    0,    0,    0],
         [  25,   16, 4753,  ...,    0,    0,    0],
         [  25,   16,  280,  ...,    0,    0,    0]]),
 tensor([[  16,  477,  200,  ...,    0,    0,    0],
         [  16, 1489,  625,  ...,    0,    0,    0],
         [  16, 1268,  382,  ...,   63,    6,    9],
         ...,
         [  16, 2002,  469,  ...,    0,    0,    0],
         [  16, 4753,   26,  ...,    0,    0,    0],
         [  16,  280,   13,  ...,    0,    0,    0]]))

In [58]:
example_index = 0  # Index of the example sentence you want to examine
example_tokens = text_ds[example_index]
print(example_tokens)

['recipe', 'for', 'lentil', ',', 'apple', ',', 'and', 'turkey', 'wrap', '|', '1', '.', 'place', 'the', 'stock', ',', 'lentils', ',', 'celery', ',', 'carrot', ',', 'thyme', ',', 'and', 'salt', 'in', 'a', 'medium', 'saucepan', 'and', 'bring', 'to', 'a', 'boil', '.', 'reduce', 'heat', 'to', 'low', 'and', 'simmer', 'until', 'the', 'lentils', 'are', 'tender', ',', 'about', '30', 'minutes', ',', 'depending', 'on', 'the', 'lentils', '.', '(', 'if', 'they', 'begin', 'to', 'dry', 'out', ',', 'add', 'water', 'as', 'needed', '.', ')', 'remove', 'and', 'discard', 'the', 'thyme', '.', 'drain', 'and', 'transfer', 'the', 'mixture', 'to', 'a', 'bowl', 'let', 'cool', '.', '2', '.', 'fold', 'in', 'the', 'tomato', ',', 'apple', ',', 'lemon', 'juice', ',', 'and', 'olive', 'oil', '.', 'season', 'with', 'the', 'pepper', '.', '3', '.', 'to', 'assemble', 'a', 'wrap', ',', 'place', '1', 'lavash', 'sheet', 'on', 'a', 'clean', 'work', 'surface', '.', 'spread', 'some', 'of', 'the', 'lentil', 'mixture', 'on', 'the

## Building the LSTM model

In [61]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.log_softmax = nn.LogSoftmax(dim=2)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x)
        return self.log_softmax(x)

In [62]:
model = LSTMModel(VOCAB_SIZE, EMBEDDING_DIM, N_UNITS)

print(model)

LSTMModel(
  (embedding): Embedding(10000, 100)
  (lstm): LSTM(100, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=10000, bias=True)
  (log_softmax): LogSoftmax(dim=2)
)


In [67]:
import torch.optim as optim
from torch.nn.functional import nll_loss
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

# Define the loss function and optimizer
loss_fn = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())

# Define the number of epochs
EPOCHS = 10  # Replace with your value

# Define a function to generate text
def generate_text(start_prompt, max_tokens, temperature):
    start_tokens = [vocab[token] for token in start_prompt.split()]
    for _ in range(max_tokens):
        x = torch.tensor([start_tokens]).to(device)
        y = model(x)
        probs = torch.exp(y[0][-1])  # Convert log probabilities to probabilities
        sample_token = torch.multinomial(probs, num_samples=1, replacement=True)
        start_tokens.append(sample_token.item())
        if sample_token.item() == vocab['<eos>']:
            break
    generated_text = ' '.join([vocab.get_itos()[token] for token in start_tokens])
    print(f"\ngenerated text:\n{generated_text}\n")

# Move the model to the device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'mps')
model.to(device)

# Create a SummaryWriter
writer = SummaryWriter()

# Training loop
for epoch in tqdm(range(EPOCHS)):
    for i, (x, y) in enumerate(text_loader):
        # Move the data to the device
        x = x.to(device)
        y = y.to(device)

        # Forward pass
        y_pred = model(x)

        # Compute the loss
        loss = loss_fn(y_pred.reshape(-1, VOCAB_SIZE), y.reshape(-1))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss every 100 batches
        if i % 100 == 0:
            print(f'Epoch {epoch}, Batch {i}, Loss: {loss.item()}')

        # Log loss and perplexity to TensorBoard
        writer.add_scalar('Loss', loss.item(), epoch * len(text_loader) + i)
        writer.add_scalar('Perplexity', torch.exp(loss).item(), epoch * len(text_loader) + i)

    # Generate text at the end of each epoch
    generate_text("recipe for", max_tokens=100, temperature=1.0)

# Close the SummaryWriter
writer.close()

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0, Batch 0, Loss: 4.714126110076904


In [ ]:
# save the model
torch.save(model.state_dict(), 'lstm_text_genv1.pth')